In [1]:
import os
import re
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm as log_progress
from pandas_profiling import ProfileReport
import plotly.express as px

## Extração dos preços (valor da cota) do relatório dos fundos (CVM)
___

In [2]:
import warnings
warnings.filterwarnings('ignore')

BASE_PATH = "E:\\data-science\\raw-data"
RES_PATH = "E:\\data-science\\datalake"


def extract(path, sep=";"):
    df = pd.DataFrame()
    for file in log_progress(os.listdir(path)):
        sm_df = pd.read_csv(f"{path}\\{file}", sep=sep)
        df = pd.concat([df, sm_df])
    return df

In [3]:
funds = extract(f"{BASE_PATH}\\cvm\\funds-report\\ia")
print(f"Coletados {funds.shape[0]} preços de fundos")

  0%|          | 0/21 [00:00<?, ?it/s]

Coletados 9602508 preços de fundos


In [4]:
funds["DT_COMPTC"] = pd.to_datetime(funds["DT_COMPTC"])

In [5]:
def keep_numbers(text):
    return re.sub("[^0-9]", "", text)

funds["id"] = funds["CNPJ_FUNDO"].apply(keep_numbers)

In [6]:
print(f"{funds['id'].unique().shape[0]} fundos encontrados")

26623 fundos encontrados


In [7]:
cols = {
    "id": "id",
    "TP_FUNDO": "type",
    "DT_COMPTC": "date",
    "VL_QUOTA": "close"
}
funds = funds[cols.keys()].rename(columns=cols)
funds.head()

,id,type,date,close
0,00017024000153,FI,2021-01-04,27.503336
1,00017024000153,FI,2021-01-05,27.503455
2,00017024000153,FI,2021-01-06,27.503196
3,00017024000153,FI,2021-01-07,27.503352
4,00017024000153,FI,2021-01-08,27.503710


In [8]:
funds.to_parquet(f"{RES_PATH}/funds_prices.snappy.parquet")